In [1]:
import torch
import brt.nn as nn
from brt.router._random import RandomScatterRouter, RandomGatherRouter


class MoE(nn.Module):
    def __init__(self):
        super().__init__()
        self.scatter_router = RandomScatterRouter(dst_num=2)
        self.expert1 = nn.Linear(10, 10)
        self.expert2 = nn.Linear(10, 10)
        self.gather_router = RandomGatherRouter(dst_num=2)

    def forward(self, x):
        route_results = self.scatter_router(x)
        x_0 = self.expert1(route_results[0])
        x_1 = self.expert2(route_results[1])
        x = self.gather_router([x_0, x_1])
        return x


class ThorMoE(nn.Module):
    def __init__(
        self,
    ):
        super().__init__()
        self.moe = MoE()

    def forward(self, x):
        return self.moe(x)


thor_moe = ThorMoE()
x = torch.arange(0, 30, dtype=torch.float32).view(3, 10)
x = thor_moe(x)
print(x)


FlowTensor:
data: tensor([[  4.2140,   0.9085,   1.5545,   1.7966,  -4.5195,   1.5102,  -2.8344,
           3.6941,   4.4830,  -4.5715],
        [ 11.3935,   7.3480,   7.7715,   5.5238, -10.5437,   4.6983,  -6.4914,
          11.6486,  13.1363, -16.0143],
        [ 18.5730,  13.7875,  13.9886,   9.2509, -16.5679,   7.8865, -10.1483,
          19.6031,  21.7896, -27.4572]])
tags: tensor([[0],
        [1],
        [2]])
load: 3
